In [1]:
!pip install gradio
!pip install transformers
!pip install evaluate
import gradio as gr
import torch
import torchtext
import csv
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 26.7 MB/s 
     |████████████████████████████████| 54 kB 4.2 MB/s 
     |████████████████████████████████| 278 kB 78.7 MB/s 
     |████████████████████████████████| 56 kB 6.1 MB/s 
     |████████████████████████████████| 213 kB 56.7 MB/s 
     |████████████████████████████████| 2.3 MB 53.2 MB/s 
     |████████████████████████████████| 106 kB 65.9 MB/s 
     |████████████████████████████████| 84 kB 4.6 MB/s 
     |████████████████████████████████| 84 kB 4.9 MB/s 
     |████████████████████████████████| 55 kB 4.9 MB/s 
     |████████████████████████████████| 64 kB 3.8 MB/s 
     |████████████████████████████████| 80 kB 11.8 MB/s 
     |████████████████████████████████| 68 kB 8.7 MB/s 
     |████████████████████████████████| 68 kB 8.1 MB/s 
     |████████████████████████████████| 68 kB 8.5 MB/s 
     |████████████████████████████████| 68 kB 8.1

In [17]:
classes = {
    '<4': 0,
    '4.0': 1,
    '4.5': 2,
    '5.0': 3,
    '5.5': 4,
    '6.0': 5,
    '6.5': 6,
    '7.0': 7,
    '7.5': 8,
    '8.0': 9,
    '8.5': 10,
    '9.0': 11
}

scores = torch.tensor([3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9])

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

tokenizer.pad_token = tokenizer.eos_token

In [6]:
from transformers import GPT2Model


def custom_activation(inputs):
  inputs[inputs<3.5] = 3.5
  inputs[inputs>9] = 9
  return inputs

class GPT2_with_regressor(torch.nn.Module):
  def __init__(self, embd_size):
    super(GPT2_with_regressor, self).__init__()
    self.encoder = GPT2Model.from_pretrained("gpt2-medium")
    self.regressor = torch.nn.Linear(embd_size, 1)

  def forward(self, input, last_idxes):
    outputs = self.encoder(**input).last_hidden_state
    hidden_states = []
    for i in range(len(last_idxes)):
      hidden_states.append(outputs[i, last_idxes[i], :][None, :])
    hidden_states = torch.cat(hidden_states, dim=0)
    outputs = self.regressor(hidden_states)
    return outputs

In [ ]:
model = GPT2_with_regressor(1024)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/ECE1786/project/gpt2_regression_10.pt"))
model.encoder.config.pad_token_id = tokenizer.pad_token_id
model.eval()

In [20]:
scores = scores.to(device)
def predict(topic, content):
    input = topic.strip() + " " + content.strip()
    encodings_dict = tokenizer(input, truncation=True, max_length=512, padding="max_length")
    input_ids = torch.tensor(encodings_dict['input_ids'])
    attention_mask = torch.tensor(encodings_dict['attention_mask']).unsqueeze(0)

    eot_idx = len(input_ids) - 1
    for i in range(len(input_ids)):
      id = input_ids[i]
      if id == tokenizer.encode("<|endoftext|>")[0]:
          eot_idx = i - 1
          break
    input_ids = input_ids.unsqueeze(0)
    eot_idx = [eot_idx]
    x = {
          "input_ids": input_ids,
          "attention_mask": attention_mask
    }
    batch = {k: v.to(device) for k, v in x.items()}
    outputs = model(batch, eot_idx)
    outputs = custom_activation(outputs).squeeze()
    pred_idx = torch.argmin(abs(scores - outputs))
    prediction = scores[pred_idx]
    print(prediction)
    return prediction
    # predictions = []
    # if len(outputs.size()) == 0:
    #       pred_idx = torch.argmin(abs(scores - outputs))
    #       predictions.append(pred_idx)
    # else:
    #   for i in range(outputs.size(0)):
    #       pred_idx = torch.argmin(abs(scores - outputs[i]))
    #       predictions.append(pred_idx)
    
    # # predictions = torch.argmax(logits, dim=-1)
    # # print(scores[predictions[0]])
    # return scores[predictions[0]]

demo = gr.Interface(
    fn=predict,
    inputs=["text", "text"],
    outputs=["text"],
)
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

tensor(7.5000, device='cuda:0')
tensor(5.5000, device='cuda:0')
tensor(7.5000, device='cuda:0')
Keyboard interruption in main thread... closing server.
